In [1]:
!pip install prophet

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import prophet as fbp
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Use fivethirtyeight plot style
#plt.style.use('fivethirtyeight')

In [ ]:
# **Data Loading and Preprocessing**
# We will be using amazon share price data which can be downloaded from YAHOO finance website.
file_path = '/content/Share-Price-Forecasting-Using-Facebook-Prophet---Dataset(1).csv'
df = pd.read_csv(file_path)
# Display first few rows of the data
df.head()

In [ ]:
# Check for missing values
print(df.isnull().sum())

In [ ]:
# **Data Visualization**
import matplotlib.dates as mdates
plt.plot(df['Date'], df['Adj Close'], label="Full Data")
#plt.xlabel('Date')
plt.xticks(rotation=45)

# Improve date formatting
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Format as YYYY-MM-DD
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=6))  # Show every 6 months


In [ ]:
# Prepare data for Prophet

df[['ds', 'y']] = df[['Date', 'Adj Close']]# add two columnsin dataframe having values as Date and Adj Close

df1 = df[['ds', 'y']]# Subset two columns from data frame

df1.head()

In [ ]:
#import matplotlib.pyplot as plt

#plt.figure(figsize=(12,5))
plt.plot(df['ds'], df['y'], label="Full Data")
plt.axvline(x="2019-07-21", color='red', linestyle='--', label="Split Date")
plt.legend()
plt.show()

In [ ]:
# **Data Splitting**
# Split data into training and testing sets
split_date = "2019-07-21"

df_train = df.loc[df.ds <= split_date].copy()
df_test = df.loc[df.ds > split_date].copy()


In [ ]:
split_date = "2019-07-21"

df1_train = df1.loc[df1.ds <= split_date].copy()
df1_test = df1.loc[df1.ds > split_date].copy()


In [ ]:
# **Model Training and Forecasting (Univariate)**
# Instantiate prophet
model1 = fbp.Prophet(changepoint_prior_scale=0.05, seasonality_prior_scale=10)
# fit the training data
model1.fit(df1_train)


In [ ]:
# Generate forecasts
forecast1 = model1.predict(df1_test)
forecast1.tail()


In [ ]:
# Visualize forecast
model1.plot(forecast1)

In [ ]:
# plot graphs of different components:
model1.plot_components(forecast1)

In [ ]:
# **Model Evaluation (Univariate)**
from prophet.diagnostics import cross_validation, performance_metrics

# Perform time-series cross-validation
df_cv = cross_validation(model1, horizon='30 days', period='90 days', initial='365 days')

# Evaluate performance
df_p = performance_metrics(df_cv)
print(df_p[['horizon', 'rmse', 'mape']])


In [ ]:
# Calculate and print error metrics
print("Mean Squared Error (MSE):", mean_squared_error(y_true = df1_test["y"], y_pred = forecast1['yhat']))
print("Mean Absolute Error (MAE):", mean_absolute_error(y_true = df1_test["y"], y_pred = forecast1['yhat']))


In [ ]:
# Calculate and print Mean Absolute Percentage Error (MAPE)
def mean_abs_perc_err(y_true, y_pred):
	return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print("Mean Absolute % Error (MAPE): ", mean_abs_perc_err(y_true = np.asarray(df1_test["y"]), y_pred = np.asarray(forecast1['yhat'])))


In [ ]:
# **Model Training and Forecasting (Multivariate)**
# Instantiate prophet using additional regressors
model = fbp.Prophet(changepoint_prior_scale=0.05, seasonality_prior_scale=10)
# fit the training data using additional regressors
model.add_regressor('Volume')
model.add_regressor('High')
model.add_regressor('Low')
# fit the training data
model.fit(df_train)

In [ ]:
# Generate forecasts
forecast = model.predict(df_test)
forecast.tail()

In [ ]:
# Visualize forecast
model.plot(forecast)


In [ ]:
# plot graphs of different components:
model.plot_components(forecast)


In [ ]:
# **Model Evaluation (Multivariate)**
from prophet.diagnostics import cross_validation, performance_metrics

# Perform time-series cross-validation
df_cv = cross_validation(model1, horizon='30 days', period='90 days', initial='365 days')

# Evaluate performance
df_p = performance_metrics(df_cv)
print(df_p[['horizon', 'rmse', 'mape']])



In [ ]:
# Calculate and print error metrics
print("Mean Squared Error (MSE):", mean_squared_error(y_true = df_test["y"], y_pred = forecast['yhat']))
print("Mean Absolute Error (MAE):", mean_absolute_error(y_true = df_test["y"], y_pred = forecast['yhat']))


In [ ]:
# Calculate and print Mean Absolute Percentage Error (MAPE)
def mean_abs_perc_err(y_true, y_pred):
	return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print("Mean Absolute % Error (MAPE): ", mean_abs_perc_err(y_true = np.asarray(df_test["y"]), y_pred = np.asarray(forecast['yhat'])))